In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.runnables import RunnableParallel, RunnableSequence, RunnableLambda, RunnablePassthrough
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain_text_splitters  import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv

load_dotenv()

True

# Load Document

In [4]:
video_id = 't8aSqlC_Duo'
ytt_api = YouTubeTranscriptApi()

try:
    transcript_list =  ytt_api.fetch(video_id)

    transcript = " ".join(chunk.text for chunk in transcript_list)
    print(transcript)
except Exception as e:
    print(e)
    # print("Transcripts are disabled for this video.")

Recently, I've been itching to build something 
with C, both to improve my skills with the   language, but also to actually just make a 
project. I knew I wanted to make something   terminal focused because I wanted to play around 
with making ANSI codes for coloring terminal   text. I also wanted to work with images in C. I do 
have some experience with computer vision stuff,   but not really in a low-level context. I decided 
to write a little project that would take in an   image and then print it out to the terminal using 
ASCII characters and terminal colors. We've also got   a little secret sauce going on to highlight the 
edges of the image. In order to print an image,   you first need to load it in from storage. I 
initially wanted to write my own bitmap parser,   which would have been simple enough, but 
I wanted this to work for any file type,   and parsing JPEG and PNG files seemed a 
little bit trickier. After some digging,   I found this library called STB image that looke

# Text Splitter

In [5]:
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100)
chunks = splitter.create_documents([transcript])

In [6]:
len(chunks)

27

In [7]:
print(chunks[5].page_content)

really simple to use. I also appreciated that it   was all in a single header file. So setting it 
up was as simple as downloading the file and then   writing an include statement. So how does it work?


In [9]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")
vectorstore = FAISS.from_documents(chunks, embeddings)

In [10]:
vectorstore.index_to_docstore_id

{0: '9b690c88-5788-43c3-94bf-367165567e6e',
 1: '326631fd-4169-4b43-b2e2-8020e06a90b6',
 2: '5f5f240a-f189-43f3-8c8b-8594d7afd978',
 3: 'a849811f-d58d-4767-b817-8996cc41f3fb',
 4: '7c9bd200-adf8-436e-a8a0-6742f014a2e6',
 5: '1bc08967-23aa-4b94-aa20-9e3bda9b1dd1',
 6: 'c5aa1f07-1a62-4c17-a130-288a59270515',
 7: '2622b3aa-59b7-49da-bf8c-232e20c64a82',
 8: 'b7ed5a52-ed00-43e2-af3b-a6c091826061',
 9: '40d1b2e0-f2fa-4e63-b1cb-2cdec4332143',
 10: '3bbab34a-20b8-4f96-95a1-c381ae0b5773',
 11: 'd10f4e78-24b0-4d4d-8e24-2b318a6fc9f4',
 12: 'bc6cf8d4-9102-468a-a8b6-ad345c2c172c',
 13: '7c5057a4-83ce-4bc4-af8b-aabf4015d9c2',
 14: '02188667-41e2-4411-b276-8fda5326eda6',
 15: 'b0b846a4-37e7-4a7d-8ff1-6500836d3572',
 16: '2b796d4a-bd25-4aac-b28c-b9c310ecafca',
 17: '03c4001e-ee79-48ad-9a1e-7db7093e779b',
 18: 'c567f8e1-5006-4864-83b2-d3d4ec76295a',
 19: 'a106c22b-eaae-4f8d-bfa8-c69155757898',
 20: '4597edd1-4757-4987-a963-4c744002a1b9',
 21: '803541fa-e7ae-4a3c-a138-237e17f1d67a',
 22: '5b5d218f-9c4f-

In [11]:
vectorstore.get_by_ids(['2770b3a5-e5ad-430d-947d-08cf68eef6dc'])

[]

In [12]:
retriver = vectorstore.as_retriever(search_type='similarity',search_kwargs={"k": 2})

In [13]:
retriver


VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001B8D95ECA40>, search_kwargs={'k': 2})

In [14]:
retriver.invoke("how can i color a text in terminal")

[Document(id='bc6cf8d4-9102-468a-a8b6-ad345c2c172c', metadata={}, page_content="for some images, it's quite hard to see what's\xa0\xa0 going on for others. An easy fix here is to add\xa0\nextra contrast with colors. We can add colors to\xa0\xa0 our terminal text by using ANSI escape sequences."),
 Document(id='02188667-41e2-4411-b276-8fda5326eda6', metadata={}, page_content="to move the cursor to the next line. Conveniently,\xa0\xa0 there's a set of these which we can use to change\xa0\nthe color of the text, and they're quite well\xa0\xa0 supported. How do we decide what color to use for")]

In [15]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",  # or another supported model name
    temperature=0.2,
    max_tokens=None,  # use default or set a limit
    timeout=None
)

In [16]:
prompt = PromptTemplate(
    template = """
    You are a helpful assistant that answers questions ONLY from the provided context. If the context is insufficient, just say you dont know.
    The context is provided below.
    {context}
    Question: {question}
    Answer:
    """,
    input_variables=["context", "question"]
)

In [17]:
question = "how can i color a text in terminal"

retriver_docs = retriver.invoke(question)

In [18]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
context = format_docs(retriver_docs)

In [19]:
print(context)

for some images, it's quite hard to see what's   going on for others. An easy fix here is to add 
extra contrast with colors. We can add colors to   our terminal text by using ANSI escape sequences.

to move the cursor to the next line. Conveniently,   there's a set of these which we can use to change 
the color of the text, and they're quite well   supported. How do we decide what color to use for


In [20]:
final_prompt = prompt.invoke({"context": context, "question": question})
print(final_prompt)

text="\n    You are a helpful assistant that answers questions ONLY from the provided context. If the context is insufficient, just say you dont know.\n    The context is provided below.\n    for some images, it's quite hard to see what's\xa0\xa0 going on for others. An easy fix here is to add\xa0\nextra contrast with colors. We can add colors to\xa0\xa0 our terminal text by using ANSI escape sequences.\n\nto move the cursor to the next line. Conveniently,\xa0\xa0 there's a set of these which we can use to change\xa0\nthe color of the text, and they're quite well\xa0\xa0 supported. How do we decide what color to use for\n    Question: how can i color a text in terminal\n    Answer:\n    "


In [21]:
answer = llm.invoke(final_prompt)

In [22]:
answer.content

'You can add colors to your terminal text by using ANSI escape sequences.'

In [23]:
parallel_chain = RunnableParallel(
    {    
    'context' : retriver | RunnableLambda(format_docs),
    'question' : RunnablePassthrough()
    }
)

In [24]:
parallel_chain.invoke(question)

{'context': "for some images, it's quite hard to see what's\xa0\xa0 going on for others. An easy fix here is to add\xa0\nextra contrast with colors. We can add colors to\xa0\xa0 our terminal text by using ANSI escape sequences.\n\nto move the cursor to the next line. Conveniently,\xa0\xa0 there's a set of these which we can use to change\xa0\nthe color of the text, and they're quite well\xa0\xa0 supported. How do we decide what color to use for",
 'question': 'how can i color a text in terminal'}

In [25]:
main_chain = parallel_chain | prompt | llm | StrOutputParser()

In [26]:
main_chain.invoke(question)

'You can add colors to your terminal text by using ANSI escape sequences.'